In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
os.chdir("C:/Users/Administrator/Downloads")

census = pd.read_csv("census_mini.csv", sep=";")
#resultant DF is capitals for each occ by year. 3 dimensional. 
#I can do corelations for each occupation through time, for occupations in one slice of time

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (3,5,6,7,10,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
census[census["YEAR"]==1900]

,YEAR,SAMPLE,SERIAL,CBSERIAL,PERNUM,PERWT,EDUC,EDUCD,OCC,OCC1950,OCC1990,INCTOT,FTOTINC,PRESGL,PRENT
0,1900,190001,1,,1,"19,71",,,190,523,,,,"45,3",
1,1900,190001,1,,2,"19,71",,,999,999,,,,0,
2,1900,190001,1,,3,"19,71",,,999,999,,,,0,
3,1900,190001,1,,4,"19,71",,,170,684,,,,"24,9",
4,1900,190001,1,,5,"19,71",,,170,675,,,,"24,9",
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3852847,1900,190001,871997,,2,"20,84",,,999,999,,,,0,
3852848,1900,190001,871997,,3,"20,84",,,999,999,,,,0,
3852849,1900,190001,871997,,4,"20,84",,,999,999,,,,0,
3852850,1900,190001,871997,,5,"20,84",,,999,999,,,,0,


In [8]:
occ_cols = {"OCC1950":occ1950_map, "OCC1990":occ1990_map}
occ_col = "OCC1990"
occ_map = {}
capitals = {"EDUC":"Education", "PRESGL":"Status", "FTOTINC":"Affluence"}
#+ INCTOT and FTOTINC and CLASSWKR/OCCSCORE for affluence, EDUC>HIGRADE>EDSCORE90 for education, PRENT also for status
#EDUC, PRESGL (though tied to 60's) useable as is
#exclude on INCs: 9999999 = N/A, EDUC: 00 (N/A or no schooling)
#OCC: https://usa.ipums.org/usa/volii/occ1990.shtml & https://usa.ipums.org/usa-action/variables/OCC1950#codes_section
#years = list(full_census["YEAR"].unique())
year = 1990
#census = full_census#[full_census["YEAR"] == year]
#del full_census
#for occ_col, occ_map in occ_cols.items():
 #   census[occ_col] = census[occ_col].replace([int(i) for i in list(occ_map.keys())], list(occ_map.values()))

census = census[census["EDUC"] != "0"]
census = census[census["EDUC"] != 0]
census = census[census["INCTOT"] != "9999999"]
census = census[census["FTOTINC"] != "9999999"]

In [9]:
for col in ["EDUC", "INCTOT", "FTOTINC","PERWT", "PRENT"]:
    print(col)
    census[col] = census[col].replace(to_replace=r' ', value='', regex=True)
    census = census[census[col] != ""]
    census[col] = census[col].replace(to_replace=r',', value='.', regex=True).astype(float)

EDUC
INCTOT
FTOTINC
PERWT
PRENT


In [13]:
years = list(census["YEAR"].unique())

In [14]:
#STILL NEED TO DO:  na removing, integrate garg occ mapping
def agg_fun(x):
    d = {}
    d['EDUC_mean'] = np.average(x['EDUC'], weights=x["PERWT"])
    d['PRESGL_mean'] = np.average(x['PRESGL'], weights=x["PERWT"])
    #d['PRENT_mean'] = np.average(x['PRENT'], weights=x["PERWT"])
    d['INCTOT_mean'] = np.average(x['INCTOT'], weights=x["PERWT"])
    d['FTOTINC_mean'] = np.average(x['FTOTINC'], weights=x["PERWT"])
    return pd.Series(d, index=['EDUC_mean', 'PRESGL_mean', 'PRENT_mean', 'FTOTINC_mean', 'INCTOT_mean'])


In [15]:
for occ_col, occ_map in occ_cols.items():
    print(occ_col)
    #census[occ_col] = census[occ_col].replace([int(i) for i in list(occ_map.keys())], list(occ_map.values()))
    for year in years[::-1]:
        print(year)
        cen = census[census["YEAR"] == year]
        try:
            #[["PRESGL"] = cen["PRESGL"].replace(to_replace=r',', value='.', regex=True).astype('float64')
            cen[["PRESGL","PRENT"]] = cen[["PRESGL","PRENT"]].replace(to_replace=r',', value='.', regex=True).astype('float64')
        except:
            pass
        agg_census = cen.groupby(occ_col).apply(agg_fun)
       # agg_census[occ_col] = agg_census[occ_col].replace([int(i) for i in list(occ_map.keys())], list(occ_map.values()))
        occ_new = {}
        for key, val in occ_map.items():
            occ_new[int(key)] = val
        agg_census.rename(index=occ_new, inplace=True)
        agg_census.to_csv("census_" + str(year) + "_" + occ_col + ".csv")
        del cen
        del agg_census

    

OCC1950
2020


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


2010
2000
1990
1980
1970
1960
1950
OCC1990
2020
2010
2000
1990
1980
1970
1960
1950


In [33]:
occ_new = {}
for key, val in occ_map.items():
    occ_new[int(key)] = val
agg_census.rename(index=occ_new, inplace=True)
agg_census

,EDUC_mean,PRESGL_mean,PRENT_mean,FTOTINC_mean,INCTOT_mean
OCC1950,,,,,
Accountants and auditors,9.837828,56.7,65.400000,158440.621475,8.395428e+04
Actors and actresses,8.358561,55.0,57.600000,235943.348622,3.870080e+04
Airplane pilots and navigators,9.248422,70.1,61.000000,225149.302006,1.605334e+05
Architects,10.168636,70.5,73.200000,194244.732856,9.578483e+04
Artists and art teachers,8.600548,56.2,52.400000,210819.770564,4.687984e+04
...,...,...,...,...,...
Fishermen and oystermen,6.318299,30.2,34.100000,579108.824402,4.818489e+04
"Gardeners, except farm and groundskeepers",5.944542,22.5,28.600000,676074.121400,2.479714e+04
"Lumbermen, raftsmen, and woodchoppers",5.796627,26.4,33.300000,348810.421784,3.651930e+04


In [ ]:
mapping = pd.read_csv("G:/My Drive/KU/Thesis/data/garg/occupatons_map.csv", index_col=None)
occu_conv = {row["1950 basis"]:row["Single words"] for row in mapping.iterrows()}
capitals = {"PRESGL": "status", "INCTOT":"affluence", "EDUC":"education", "EDUC":"cultivation"}
averages = {}
for occ in occu_conv.keys():
    averages[occ] = {}
    for capital in capitals.keys():
        averages[occ][capital + "_mean"] = census[census.loc[:,"occ1950"]==occ].loc[:,capital].mean()
        averages[occ][capital + "_median"] = census[census.loc[:,"occ1950"]==occ].loc[:,capital].median()
        #gcbs[gcbs.loc[:,el].isin([2,3])].loc[:,capital].mean()

# GBCS

In [ ]:
gcbs = pd.read_stata("G:/My Drive/KU/Thesis/data/gbcs_stata/stata11/gbcs_cleaned_eul.dta")#, convert_categoricals=False)

In [ ]:
activities_conv = {"carts":"gallery", "cbingo":"bingo", "csportw":"sport", "ctheatre":"theatre", 
                "copera":"opera", "cdance":"dance","cmags":"magazine", "cdiy":"diy", 
                "cpub":"pub", "cclassic":"classical", "cshop":"shopping", "cresta":"restaurant",
                "ctv":"tv", "cbooks":"reading", "cgig":"gig"}
music_conv = {"mmetal":"metal","mreggae":"reggae", "mjazz":"jazz", "mpop":"pop", "mfolk":"folk",
             "mclassic":"classical", "mrock":"rock", "mcountry":"country"}
conv = dict(activities_conv, **music_conv)
capitals = {"hhincome": "affluence", "educ":"education", "chigh":"cultivation"} 
act_likert_conv = {"Never":0, "Rarely":1, "Sometimes":2, "Often":3}
music_likert_conv = {"Dislike a lot":1, "Dislike":2, "Neither":3, "Like":4, "Like a lot":5}
for el in activities_conv.keys():
    gcbs.loc[:,el]=  gcbs.loc[:,el].replace(act_likert_conv)
for el in music_conv.keys():
    gcbs.loc[:,el]=  gcbs.loc[:,el].replace(music_likert_conv)

In [ ]:
#gcbs["hhincome"] = gcbs["hhincome"].apply(lambda x: 2.5+5*(x-1)) 
sorted(list(gcbs["hhincome"].unique())[:-1])
hhincome_conv = {'Over £200,000':300000,
 'Under £5,000':2500,
 '£10,000 - £14,999':12500,
 '£100,000 - £149,999':125000,
 '£15,000 - £19,999':17500,
 '£150,000 - £199,999':175000,
 '£20,000 - £24,999':22500,
 '£25,000 - £29,999':27500,
 '£30,000 - £34,999':32500,
 '£35,000 - £39,999':37500,
 '£40,000 - £44,999':42500,
 '£45,000 - £49,999':47500,
 '£5,000 - £10,000':7500,
 '£50,000 - £54,999':52500,
 '£55,000 - £59,999':57500,
 '£60,000 - £64,999':62500,
 '£65,000 - £69,999':67500,
 '£70,000 - £74,999':72500,
 '£75,000 - £79,999':77500,
 '£80,000 - £84,999':82500,
 '£85,000 - £89,999':87500,
 '£90,000 - £94,999':92500,
 '£95,000 - £99,999':97500, 
 np.nan: 0}
educ_conv = {
    '1.  Did not complete GCSE/CSE/O-levels':1,
 '2. A-levels/Scottish Highers':2,
 '3. Completed GCSE/CSE/O-levels':3,
 '4. Completed post-16 vocational course':4,
 '5. I am still in education':5,
 '6. Postgraduate degree':6,
 '7. Undergraduate degree':7
}

gcbs.loc[:,"educ"]=  gcbs.loc[:,"educ"].replace(educ_conv)
gcbs.loc[:,"hhincome"]=  gcbs.loc[:,"hhincome"].replace(hhincome_conv)

In [ ]:
gcbs.loc[:,"hhincome"].unique()

In [ ]:
averages = {el : {} for el in conv.keys()}
for el in activities_conv.keys():
    averages[el] = {}
    for capital in capitals.keys():
        averages[el][capital + "_mean"] = gcbs[gcbs.loc[:,el].isin([2,3])].loc[:,capital].mean()
        averages[el][capital + "_median"] = gcbs[gcbs.loc[:,el].isin([2,3])].loc[:,capital].median()
                                                                         
for el in music_conv.keys():
    averages[el] = {}
    for capital in capitals.keys():
        averages[el][capital + "_mean"] = gcbs[gcbs.loc[:,el].isin([4,5])].loc[:,capital].mean()
        averages[el][capital + "_median"] = gcbs[gcbs.loc[:,el].isin([4,5])].loc[:,capital].median()   

In [ ]:
df
df = pd.DataFrame(averages)
df = df.rename_axis("activity")
#df.set_index("activity")
dft = df.T
#gcbs.shape


In [ ]:
dft

In [ ]:
dft.to_csv(f"G:/My Drive/KU/Thesis/data/survey_data/gbcs_averages.csv")
pd.read_csv("G:/My Drive/KU/Thesis/data/survey_data/gbcs_averages.csv", index_col= 0)#,index_col="activity")

In [ ]:
gss = pd.read_stata("G:/My Drive/KU/Thesis/data/GSS_stata/gss7221_r1b.dta", convert_categoricals=False)

In [ ]:
activities = ["attsprts", "visitart", "makeart", "autorace", "camping", "garden", "dance", "gomusic", "huntfish", "perform", "dosports", "seemovie", "usevcr", "plymusic", "readfict", "popmusic", "drama", "relart", "volarts"] 
music_genres = ["bigband","blugrass", "country", "blues", "musicals", "classicl", "folk", "gospel", "jazz", "latin", "moodeasy", "newage", "opera", "rap", "reggae", "conrock", "oldies", "hvymetal"]
cont = ["gamegn"]
domains = activities + music_genres
capitals = ["educ", "prestg80", "rincome", "wealth", "class"]
gss[activities] = gss[activities].apply(lambda x: abs(x-2))
 #class: 1 lower, 2 working, 3 middle, 4 upper, 5 no
gss[["educ", "prestige", "rincome", "wealth"]].head() #occ, occ80, occ10 industry, rincome77, rincome82, rincome86, rincome91, rincome98

#1= yes, 2= no, change to 1 = yes, 0 = no
means = {activity: {cap + "_mean": 0 for cap in capitals} for activity in domains}
medians = {activity: {cap + "_median": 0 for cap in capitals} for activity in domains}
averages = means  | medians
#gss[activities].describe()
for activity in domains:
    df = gss[gss[activity]== 1]
    print(activity, gss[-gss[activity].isna()]["year"].unique())
    for cap in capitals:
        averages[activity][cap + "_median"] = df[cap].median()
        averages[activity][cap + "_mean"] = df[cap].mean()
#for cap in capitals:
 #   print(cap, gss[-gss[cap].isna()]["year"].unique())
#gss[capitals].describe()
df = pd.DataFrame(averages).T
df.index.name = "capital"
df.to_csv(f"G:/My Drive/KU/Thesis/data/survey_data/gss_averages.csv")
pd.read_csv(f"G:/My Drive/KU/Thesis/data/survey_data/gss_averages.csv", index_col = "capital")
#music_genres_conv = {"bigband": "big band","blugrass": "bluegrass", "country": "country", "blues": "blues", "musicals": "musicals", "classicl": "classical", "folk": "folk", "gospel": "gospel", "jazz": "jazz", "latin": ["salsa", "samba", "bachata", "kilomba", "tango", "rumba", "bolero"], "moodeasy": "easy-listening", "newage": ["new-age", "newage"], "opera": "opera", "rap": "rap", "reggae": "reggae", "conrock": "rock", "oldies": "oldies", "hvymetal": "heavy-metal"}
#activities_conv = {"attsprts":"sports", "visitart":"exhibition", "makeart":"art", "autorace":"autosport", "camping":"camping", "garden":"gardening", "dance":["dancing","dance"], "gomusic":["concert", "gig"] "huntfish":["hunting","fishing"], "perform":[], "dosports":["sports", "sport"], "seemovie":[], "usevcr":[], "plymusic":[], "readfict":"reading", "popmusic":"pop", "drama":[], "relart":[], "volarts":[]}
#capitals = ["educ":"education", "prestg80", "rincome", "wealth", "class"]

In [3]:
occ1990_map = {"000": 'N/A (not applicable)',
 '003': 'Legislators',
 '004': 'Chief executives and general administrators, public administration',
 '005': 'Administrators and officials, public administration',
 '006': 'Administrators, protective services',
 '007': 'Financial managers',
 '008': 'Personnel and labor relations managers',
 '009': 'Purchasing managers',
 '013': 'Managers, marketing, advertising, and public relations',
 '014': 'Administrators, education and related fields',
 '015': 'Managers, medicine and health',
 '016': 'Postmasters and mail superintendents',
 '017': 'Managers, food serving and lodging establishments',
 '018': 'Managers, properties and real estate',
 '019': 'Funeral directors',
 '021': 'Managers, service organizations, n.e.c.',
 '022': 'Managers and administrators, n.e.c. ',
 '023': 'Accountants and auditors',
 '024': 'Underwriters',
 '025': 'Other financial officers',
 '026': 'Management analysts',
 '027': 'Personnel, training, and labor relations specialists',
 '028': 'Purchasing agents and buyers, farm products',
 '029': 'Buyers, wholesale and retail trade, except farm products',
 '033': 'Purchasing agents and buyers, n.e.c.',
 '034': 'Business and promotion agents',
 '035': 'Construction inspectors',
 '036': 'Inspectors and compliance officers, except construction',
 '037': 'Management related occupations, n.e.c. ',
 '043': 'Architects',
 '044': 'Aerospace',
 '045': 'Metallurgical and materials',
 '046': 'Mining',
 '047': 'Petroleum',
 '048': 'Chemical',
 '049': 'Nuclear',
 '053': 'Civil',
 '054': 'Agricultural',
 '055': 'Electrical and electronic',
 '056': 'Industrial',
 '057': 'Mechanical',
 '058': 'Marine and naval architects',
 '059': 'Engineers, n.e.c.',
 '063': 'Surveyors and mapping scientists ',
 '064': 'Computer systems analysts and scientists',
 '065': 'Operations and systems researchers and analysts',
 '066': 'Actuaries',
 '067': 'Statisticians',
 '068': 'Mathematical scientists, n.e.c. ',
 '069': 'Physicists and astronomers',
 '073': 'Chemists, except biochemists',
 '074': 'Atmospheric and space scientists',
 '075': 'Geologists and geodesists',
 '076': 'Physical scientists, n.e.c.',
 '077': 'Agricultural and food scientists',
 '078': 'Biological and life scientists',
 '079': 'Forestry and conservation scientists',
 '083': 'Medical scientists ',
 '084': 'Physicians',
 '085': 'Dentists',
 '086': 'Veterinarians',
 '087': 'Optometrists',
 '088': 'Podiatrists',
 '089': 'Health diagnosing practitioners, n.e.c. ',
 '095': 'Registered nurses',
 '096': 'Pharmacists',
 '097': 'Dietitians',
 '098': 'Respiratory therapists',
 '099': 'Occupational therapists',
 '103': 'Physical therapists',
 '104': 'Speech therapists',
 '105': 'Therapists, n.e.c.',
 '106': "Physicians' assistants ",
 '113': 'Earth, environmental, and marine science teachers',
 '114': 'Biological science teachers',
 '115': 'Chemistry teachers',
 '116': 'Physics teachers',
 '117': 'Natural science teachers, n.e.c.',
 '118': 'Psychology teachers',
 '119': 'Economics teachers',
 '123': 'History teachers',
 '124': 'Political science teachers',
 '125': 'Sociology teachers',
 '126': 'Social science teachers, n.e.c.',
 '127': 'Engineering teachers',
 '128': 'Mathematical science teachers',
 '129': 'Computer science teachers',
 '133': 'Medical science teachers',
 '134': 'Health specialties teachers',
 '135': 'Business, commerce, and marketing teachers',
 '136': 'Agriculture and forestry teachers',
 '137': 'Art, drama, and music teachers',
 '138': 'Physical education teachers',
 '139': 'Education teachers',
 '143': 'English teachers',
 '144': 'Foreign language teachers',
 '145': 'Law teachers',
 '146': 'Social work teachers',
 '147': 'Theology teachers',
 '148': 'Trade and industrial teachers',
 '149': 'Home economics teachers',
 '153': 'Teachers, postsecondary, n.e.c.',
 '154': 'Postsecondary teachers, subject n.s. ',
 '155': 'Teachers, prekindergarten and kindergarten',
 '156': 'Teachers, elementary school',
 '157': 'Teachers, secondary school',
 '158': 'Teachers, special education',
 '159': 'Teachers, n.e.c.',
 '163': 'Counselors, educational and vocational ',
 '164': 'Librarians',
 '165': 'Archivists and curators ',
 '166': 'Economists',
 '167': 'Psychologists',
 '168': 'Sociologists',
 '169': 'Social scientists, n.e.c.',
 '173': 'Urban planners ',
 '174': 'Social workers',
 '175': 'Recreation workers',
 '176': 'Clergy',
 '177': 'Religious workers, n.e.c. ',
 '178': 'Lawyers',
 '179': 'Judges ',
 '183': 'Authors',
 '184': 'Technical writers',
 '185': 'Designers',
 '186': 'Musicians and composers',
 '187': 'Actors and directors',
 '188': 'Painters, sculptors, craft-artists, and artist printmakers',
 '189': 'Photographers',
 '193': 'Dancers',
 '194': 'Artists, performers, and related workers, n.e.c.',
 '195': 'Editors and reporters',
 '197': 'Public relations specialists',
 '198': 'Announcers',
 '199': 'Athletes ',
 '203': 'Clinical laboratory technologists and technicians',
 '204': 'Dental hygienists',
 '205': 'Health record technologists and technicians',
 '206': 'Radiologic technicians',
 '207': 'Licensed practical nurses',
 '208': 'Health technologists and technicians, n.e.c. ',
 '213': 'Electrical and electronic technicians',
 '214': 'Industrial engineering technicians',
 '215': 'Mechanical engineering technicians',
 '216': 'Engineering technicians, n.e.c.',
 '217': 'Drafting occupations',
 '218': 'Surveying and mapping technicians',
 '223': 'Biological technicians',
 '224': 'Chemical technicians',
 '225': 'Science technicians n.e.c.',
 '226': 'Airplane pilots and navigators',
 '227': 'Air traffic controllers',
 '228': 'Broadcast equipment operators',
 '229': 'Computer programmers',
 '233': 'Tool programmers, numerical control',
 '234': 'Legal assistants',
 '235': 'Technician, n.e.c. ',
 '243': 'Supervisors and proprietors, sales occupations',
 '253': 'Insurance sales occupations',
 '254': 'Real estate sales occupations',
 '255': 'Securities and financial services sales occupations',
 '256': 'Advertising and related sales occupations',
 '257': 'Sales occupations, other business services',
 '258': 'Sales engineers',
 '259': 'Sales representatives, mining, manufacturing, and wholesale',
 '263': 'Sales workers, motor vehicles and boats',
 '264': 'Sales workers, apparel',
 '265': 'Sales workers, shoes',
 '266': 'Sales workers, furniture and home furnishings',
 '267': 'Sales workers, radio, ',
 '268': 'Sales workers, hardware and building supplies',
 '269': 'Sales workers, parts',
 '274': 'Sales workers, other commodities',
 '275': 'Sales counter clerks',
 '276': 'Cashiers',
 '277': 'Street and door-to-door sales workers',
 '278': 'News vendors',
 '283': 'Demonstrators, promoters and models, sales',
 '284': 'Auctioneers',
 '285': 'Sales support occupations, n.e.c.',
 '290': 'Sales workers -- allocat',
 '303': 'Supervisors, general office',
 '304': 'Supervisors, computer equipment operators',
 '305': 'Supervisors, financial records processing',
 '306': 'Chief communications operators',
 '307': 'Supervisors; distribution, scheduling, and adjusting clerks',
 '308': 'Computer operators',
 '309': 'Peripheral equipment operators',
 '313': 'Secretaries',
 '314': 'Stenographers',
 '315': 'Typists',
 '316': 'Interviewers',
 '317': 'Hotel clerks',
 '318': 'Transportation ticket and reservation agents',
 '319': 'Receptionists',
 '323': 'Information clerks, n.e.c.',
 '325': 'Classified-ad clerks',
 '326': 'Correspondence clerks',
 '327': 'Order clerks',
 '328': 'Personnel clerks, except payroll and timekeeping',
 '329': 'Library clerks',
 '335': 'File clerks',
 '336': 'Records clerks',
 '337': 'Bookkeepers, accounting, and auditing clerks',
 '338': 'Payroll and timekeeping clerks',
 '339': 'Billing clerks',
 '343': 'Cost and rate clerks',
 '344': 'Billing, posting, and calculating machine operators',
 '345': 'Duplicating machine operators',
 '346': 'Mail preparing and paper handling machine operators',
 '347': 'Office machine operators, n.e.c.',
 '348': 'Telephone operators',
 '353': 'Communications equipment operators, n.e.c.',
 '354': 'Postal clerks except mail carriers',
 '355': 'Mail carriers, postal service',
 '356': 'Mail clerks, except postal service',
 '357': 'Messengers',
 '359': 'Dispatchers',
 '363': 'Production coordinators',
 '364': 'Traffic, shipping, and receiving clerks',
 '365': 'Stock and inventory clerks',
 '366': 'Meter readers',
 '368': 'Weighers, measurers, checkers, and samplers',
 '373': 'Expediters',
 '374': 'Material recording, scheduling, and distributing clerks, n.e.c.',
 '375': 'Insurance adjusters, examiners, and investigators',
 '376': 'Investigators and adjusters, except insurance',
 '377': 'Eligibility clerks, social welfare',
 '378': 'Bill and account collectors',
 '379': 'General office clerks',
 '383': 'Bank tellers',
 '384': 'Proofreaders',
 '385': 'Data-entry keyers',
 '386': 'Statistical clerks',
 '387': "Teachers' aides",
 '389': 'Administrative support occupations, n.e.c.',
 '390': 'Professional, technical, and kindred workers -- allocated',
 '391': 'Clerical and kindred workers -- allocat',
 '403': 'Launderers and ironers',
 '404': 'Cooks, private household',
 '405': 'Housekeepers and butlers',
 '406': 'Child care workers, private household',
 '407': 'Private household cleaners and servants',
 '408': 'Private household workers -- allocat',
 '413': 'Supervisors, firefighting and fire prevention occupations',
 '414': 'Supervisors, police and detectives',
 '415': 'Supervisors, guards',
 '416': 'Fire inspection and fire prevention occupations',
 '417': 'Firefighting occupations',
 '418': 'Police and detectives, public service',
 '423': 'Sheriffs, bailiffs, and other law enforcement officers',
 '424': 'Correctional institution officers -',
 '425': 'Crossing guards',
 '426': 'Guards and police, except public service',
 '427': 'Protective service occupations, n.e.c. ',
 '433': 'Supervisors, food preparation and service occupations',
 '434': 'Bartenders',
 '435': 'Waiters and waitresses',
 '436': 'Cooks',
 '438': 'Food counter, fountain and related occupations',
 '439': 'Kitchen workers, food preparation',
 '443': "Waiters'/waitresses' assistants",
 '444': 'Miscellaneous food preparation occupations',
 '445': 'Dental assistants',
 '446': 'Health aides, except nursing',
 '447': 'Nursing aides, orderlies, and attendants',
 '448': 'Supervisors, cleaning and building service workers',
 '449': 'Maids and housemen',
 '453': 'Janitors and cleaners',
 '454': 'Elevator operators',
 '455': 'Pest control occupations',
 '456': 'Supervisors, personal service occupations',
 '457': 'Barbers',
 '458': 'Hairdressers and cosmetologists',
 '459': 'Attendants, amusement and recreation facilities',
 '461': 'Guides',
 '462': 'Ushers',
 '463': 'Public transportation attendants',
 '464': 'Baggage porters and bellhops',
 '465': 'Welfare service aides',
 '466': 'Family child care providers',
 '467': "Early childhood teachers' assistants",
 '468': 'Child care workers, n.e.c.',
 '469': 'Personal service occupations, n.e.c. ',
 '473': 'Farmers, except horticultural',
 '474': 'Horticultural specialty farmers',
 '475': 'Managers, farms, except horticultural',
 '476': 'Managers, horticultural specialty farms ',
 '477': 'Supervisors, farm workers',
 '479': 'Farm workers',
 '480': 'Farm laborers and farm foremen -- allocated',
 '483': 'Marine life cultivation workers',
 '484': 'Nursery workers',
 '485': 'Supervisors, related agricultural occupations',
 '486': 'Groundskeepers and gardeners, except farm',
 '487': 'Animal caretakers, except farm',
 '488': 'Graders and sorters, agricultural products',
 '489': 'Inspectors, agricultural products',
 '494': 'Supervisors, forestry, and logging workers',
 '495': 'Forestry workers except logging',
 '496': 'Timber cutting and logging occupations',
 '497': 'Captains and other officers, fishing vessels',
 '498': 'Fishers',
 '499': 'Hunters and trappers ',
 '503': 'Supervisors, mechanics and repairers ',
 '505': 'Automobile mechanics',
 '506': 'Automobile mechanic apprentices',
 '507': 'Bus, truck, and stationary engine mechanics',
 '508': 'Aircraft engine mechanics',
 '509': 'Small engine repairers',
 '514': 'Automobile body and related repairers',
 '515': 'Aircraft mechanics, except engine',
 '516': 'Heavy equipment mechanics',
 '517': 'Farm equipment mechanics',
 '518': 'Industrial machinery repairers',
 '519': 'Machinery maintenance occupations',
 '523': 'Electronic repairers, communications and industrial equipment',
 '525': 'Data processing equipment repairers',
 '526': 'Household appliance and power tool repairers',
 '527': 'Telephone line installers and repairers',
 '529': 'Telephone installers and repairers',
 '533': 'Misc. electrical and electronic equipment repairers',
 '534': 'Heating, air conditioning, and refrigeration mechanics',
 '535': 'Camera, watch, and musical instrument repairers',
 '536': 'Locksmiths and safe repairers',
 '538': 'Office machine repairers',
 '539': 'Mechanical controls and valve repairers',
 '543': 'Elevator installers and repairers',
 '544': 'Millwrights',
 '547': 'Mechanics and repairers, n.e.c.',
 '549': 'Mechanics and repairers, n.s. ',
 '553': 'Supervisors, brickmasons, stonemasons, and tile setters',
 '554': 'Supervisors, carpenters and related workers',
 '555': 'Supervisors, electricians and power transmission installers',
 '556': 'Supervisors, painters, paperhangers, and plasterers',
 '557': 'Supervisors, plumbers, pipefitters, and steamfitters',
 '558': 'Supervisors, construction, n.e.c.',
 '563': 'Brickmasons and stonemasons',
 '564': 'Brickmason and stonemason apprentices',
 '565': 'Tile setters, hard and soft',
 '566': 'Carpet installers',
 '567': 'Carpenters',
 '569': 'Carpenter apprentices',
 '573': 'Drywall installers',
 '575': 'Electricians',
 '576': 'Electrician apprentices',
 '577': 'Electrical power installers and repairers',
 '579': 'Painters, construction and maintenance',
 '583': 'Paperhangers',
 '584': 'Plasterers',
 '585': 'Plumbers, pipefitters, and steamfitters',
 '587': 'Plumber, pipe fitter, and steamfitter apprentices',
 '588': 'Concrete and terrazzo finishers',
 '589': 'Glaziers',
 '593': 'Insulation workers',
 '594': 'Paving, surfacing, and tamping equipment operators',
 '595': 'Roofers',
 '596': 'Sheetmetal duct installers',
 '597': 'Structural metal workers',
 '598': 'Drillers, earth',
 '599': 'Construction trades, n.e.c. ',
 '613': 'Supervisors, extractive occupations',
 '614': 'Drillers, oil well',
 '615': 'Explosives workers',
 '616': 'Mining machine operators',
 '617': 'Mining occupations, n.e.c. ',
 '628': 'Supervisors, production occupations',
 '634': 'Tool and die makers',
 '635': 'Tool and die maker apprentices',
 '636': 'Precision assemblers, metal',
 '637': 'Machinists',
 '639': 'Machinist apprentices',
 '643': 'Boilermakers',
 '644': 'Precision grinders, filers, and tool sharpeners',
 '645': 'Patternmakers and model makers, metal',
 '646': 'Lay-out workers',
 '647': 'Precious stones and metals workers (jewelers)',
 '649': 'Engravers, metal',
 '653': 'Sheet metal workers',
 '654': 'Sheet metal worker apprentices',
 '655': 'Miscellaneous precision metal workers',
 '656': 'Patternmakers and model makers, wood',
 '657': 'Cabinet makers and bench carpenters',
 '658': 'Furniture and wood finishers',
 '659': 'Miscellaneous precision woodworkers',
 '666': 'Dressmakers',
 '667': 'Tailors',
 '668': 'Upholsterers',
 '669': 'Shoe repairers',
 '674': 'Miscellaneous precision apparel and fabric workers',
 '675': 'Hand molders and shapers, except jewelers',
 '676': 'Patternmakers, lay-out workers, and cutters',
 '677': 'Optical goods workers',
 '678': 'Dental laboratory and medical appliance technicians',
 '679': 'Bookbinders',
 '683': 'Electrical and electronic equipment assemblers',
 '684': 'Miscellaneous precision workers, n.e.c.',
 '686': 'Butchers and meat cutters',
 '687': 'Bakers',
 '688': 'Food batchmakers',
 '689': 'Inspectors, testers, and graders',
 '693': 'Adjusters and calibrators',
 '694': 'Water and sewage treatment plant operators',
 '695': 'Power plant operators',
 '696': 'Stationary engineers',
 '699': 'Miscellaneous plant and system operators ',
 '703': 'Lathe and turning machine set-up operators',
 '704': 'Lathe and turning machine operators',
 '705': 'Milling and planing machine operators',
 '706': 'Punching and stamping press machine operators',
 '707': 'Rolling machine operators',
 '708': 'Drilling and boring machine operators',
 '709': 'Grinding, abrading, buffing, and polishing machine operators',
 '713': 'Forging machine operators',
 '714': 'Numerical control machine operators',
 '715': 'Miscellaneous metal, plastic, stone, and glass working machine operators',
 '717': 'Fabricating machine operators, n.e.c.',
 '719': 'Molding and casting machine operators',
 '723': 'Metal plating machine operators',
 '724': 'Heat treating equipment operators',
 '725': 'Miscellaneous metal and plastic processing machine operators',
 '726': 'Wood lathe, routing, and planing machine operators',
 '727': 'Sawing machine operators',
 '728': 'Shaping and joining machine operators',
 '729': 'Nailing and tacking machine operators',
 '733': 'Miscellaneous woodworking machine operators',
 '734': 'Printing press operators',
 '735': 'Photoengravers and lithographers',
 '736': 'Typesetters and compositors',
 '737': 'Miscellaneous printing machine operators',
 '738': 'Winding and twisting machine operators',
 '739': 'Knitting, looping, taping, and weaving machine operators',
 '743': 'Textile cutting machine operators',
 '744': 'Textile sewing machine operators',
 '745': 'Shoe machine operators',
 '747': 'Pressing machine operators',
 '748': 'Laundering and dry cleaning machine operators',
 '749': 'Miscellaneous textile machine operators',
 '753': 'Cementing and gluing machine operators',
 '754': 'Packaging and filling machine operators',
 '755': 'Extruding and forming machine operators',
 '756': 'Mixing and blending machine operators',
 '757': 'Separating, filtering, and clarifying machine operators',
 '758': 'Compressing and compacting machine operators',
 '759': 'Painting and paint spraying machine operators',
 '763': 'Roasting and baking machine operators, food',
 '764': 'Washing, cleaning, and pickling machine operators',
 '765': 'Folding machine operators',
 '766': 'Furnace, kiln, and oven operators, except food',
 '768': 'Crushing and grinding machine operators',
 '769': 'Slicing and cutting machine operators',
 '773': 'Motion picture projectionists',
 '774': 'Photographic process machine operators',
 '777': 'Miscellaneous machine operators, n.e.c.',
 '779': 'Machine operators, n.s. ',
 '783': 'Welders and cutters',
 '784': 'Solderers and brazers',
 '785': 'Assemblers',
 '786': 'Hand cutting and trimming occupations',
 '787': 'Hand molding, casting, and forming occupations',
 '789': 'Hand painting, coating, and decorating occupations',
 '793': 'Hand engraving and printing occupations',
 '795': 'Miscellaneous hand working occupations ',
 '796': 'Production inspectors, checkers, and examiners',
 '797': 'Production testers',
 '798': 'Production samplers and weighers',
 '799': 'Graders and sorters, except agricultural ',
 '803': 'Supervisors, motor vehicle operators',
 '804': 'Truck drivers',
 '806': 'Driver-sales workers',
 '808': 'Bus drivers',
 '809': 'Taxicab drivers and chauffeurs',
 '813': 'Parking lot attendants',
 '814': 'Motor transportation occupations, n.e.c.',
 '815': 'Transport equipment operatives -- allocat',
 '823': 'Railroad conductors and yardmasters',
 '824': 'Locomotive operating occupations',
 '825': 'Railroad brake, signal, and switch operators',
 '826': 'Rail vehicle operators, n.e.c.',
 '828': 'Ship captains and mates, except fishing boats',
 '829': 'Sailors and deckhands',
 '833': 'Marine engineers',
 '834': 'Bridge, lock, and lighthouse tenders',
 '843': 'Supervisors, material moving equipment operators',
 '844': 'Operating engineers',
 '845': 'Longshore equipment operators',
 '848': 'Hoist and winch operators',
 '849': 'Crane and tower operators',
 '853': 'Excavating and loading machine operators',
 '855': 'Grader, dozer, and scraper operators',
 '856': 'Industrial truck and tractor equipment operators',
 '859': 'Miscellaneous material moving equipment operators',
 '864': 'Supervisors, handlers, equipment cleaners, and laborers, n.e.c.',
 '865': 'Helpers, mechanics and repairers',
 '866': 'Helpers, construction trades',
 '867': 'Helpers, surveyor',
 '868': 'Helpers, extractive occupations',
 '869': 'Construction laborers',
 '874': 'Production helpers',
 '875': 'Garbage collectors',
 '876': 'Stevedores',
 '877': 'Stock handlers and baggers',
 '878': 'Machine feeders and offbearers',
 '883': 'Freight, stock, and material handlers, n.e.c.',
 '885': 'Garage and service station related occupations',
 '887': 'Vehicle washers and equipment cleaners',
 '888': 'Hand packers and packagers',
 '889': 'Laborers, except construction',
 '890': 'Laborers, except farm -- allocat',
 '903': 'Commissioned ',
 '904': 'Non-commission',
 '905': 'Military occupation, rank not specified ',
 '909': 'Last worked'}

In [4]:
occ1950_map = {'000': 'Accountants and auditors',
 '001': 'Actors and actresses',
 '002': 'Airplane pilots and navigators',
 '003': 'Architects',
 '004': 'Artists and art teachers',
 '005': 'Athletes',
 '006': 'Authors',
 '007': 'Chemists',
 '008': 'Chiropractors',
 '009': 'Clergymen',
 '010': 'College presidents and deans',
 '012': 'Agricultural sciences',
 '013': 'Biological sciences',
 '014': 'Chemistry',
 '015': 'Economics',
 '016': 'Engineering',
 '017': 'Geology and geophysics',
 '018': 'Mathematics',
 '019': 'Medical sciences',
 '023': 'Physics',
 '024': 'Psychology',
 '025': 'Statistics',
 '026': 'Natural science (n.e.c.)',
 '027': 'Social sciences (n.e.c.)',
 '028': 'Non-scientific subjects',
 '029': 'Subject not specified',
 '031': 'Dancers and dancing teachers',
 '032': 'Dentists',
 '033': 'Designers',
 '034': 'Dietitians and nutritionists',
 '035': 'Draftsmen',
 '036': 'Editors and reporters',
 '041': 'Engineers, aeronautical',
 '042': 'Engineers, chemical',
 '043': 'Engineers, civil',
 '044': 'Engineers, electrical',
 '045': 'Engineers, industrial',
 '046': 'Engineers, mechanical',
 '047': 'Engineers, metallurgical, metallurgists',
 '048': 'Engineers, mining',
 '049': 'Engineers (n.e.c.)',
 '051': 'Entertainers (n.e.c.)',
 '052': 'Farm and home management advisors',
 '053': 'Foresters and conservationists',
 '054': 'Funeral directors and embalmers',
 '055': 'Lawyers and judges',
 '056': 'Librarians',
 '057': 'Musicians and music teachers',
 '058': 'Nurses, professional',
 '059': 'Nurses, student professional',
 '061': 'Agricultural scientists',
 '062': 'Biological scientists',
 '063': 'Geologists and geophysicists',
 '067': 'Mathematicians',
 '068': 'Physicists',
 '069': 'Miscellaneous natural scientists',
 '070': 'Optometrists',
 '071': 'Osteopaths',
 '072': 'Personnel and labor relations workers',
 '073': 'Pharmacists',
 '074': 'Photographers',
 '075': 'Physicians and surgeons',
 '076': 'Radio operators',
 '077': 'Recreation and group workers',
 '078': 'Religious workers',
 '079': 'Social and welfare workers, except group',
 '081': 'Economists',
 '082': 'Psychologists',
 '083': 'Statisticians and actuaries',
 '084': 'Miscellaneous social scientists',
 '091': 'Sports instructors and officials',
 '092': 'Surveyors',
 '093': 'Teachers (n.e.c.)',
 '094': 'Technicians, medical and dental',
 '095': 'Technicians, testing',
 '096': 'Technicians (n.e.c.)',
 '097': 'Therapists and healers (n.e.c.)',
 '098': 'Veterinarians',
 '099': 'Professional, technical and kindred workers (n.e.c.)',
 '100': 'Farmers (owners and tenants)',
 '123': 'Farm managers',
 '200': 'Buyers and department heads, store',
 '201': 'Buyers and shippers, farm products',
 '203': 'Conductors, railroad',
 '204': 'Credit men',
 '205': 'Floormen and floor managers, store',
 '210': 'Inspectors, public administration',
 '230': 'Managers and superintendents, building',
 '240': 'Officers, pilots, pursers and engineers, ship',
 '250': 'Officials and administrators (n.e.c.), public administration',
 '260': 'Officials, lodge, society, union, etc.',
 '270': 'Postmasters',
 '280': 'Purchasing agents and buyers (n.e.c.)',
 '290': 'Managers, officials, and proprietors (n.e.c.)',
 '300': 'Agents (n.e.c.)',
 '301': 'Attendants and assistants, library',
 '302': "Attendants, physician's and dentist's office",
 '304': 'Baggagemen, transportation',
 '305': 'Bank tellers',
 '310': 'Bookkeepers',
 '320': 'Cashiers',
 '321': 'Collectors, bill and account',
 '322': 'Dispatchers and starters, vehicle',
 '325': 'Express messengers and railway mail clerks',
 '335': 'Mail carriers',
 '340': 'Messengers and office boys',
 '341': 'Office machine operators',
 '342': 'Shipping and receiving clerks',
 '350': 'Stenographers, typists, and secretaries',
 '360': 'Telegraph messengers',
 '365': 'Telegraph operators',
 '370': 'Telephone operators',
 '380': 'Ticket, station, and express agents',
 '390': 'Clerical and kindred workers (n.e.c.)',
 '400': 'Advertising agents and salesmen',
 '410': 'Auctioneers',
 '420': 'Demonstrators',
 '430': 'Hucksters and peddlers',
 '450': 'Insurance agents and brokers',
 '460': 'Newsboys',
 '470': 'Real estate agents and brokers',
 '480': 'Stock and bond salesmen',
 '490': 'Salesmen and sales clerks (n.e.c.)',
 '500': 'Bakers',
 '501': 'Blacksmiths',
 '502': 'Bookbinders',
 '503': 'Boilermakers',
 '504': 'Brickmasons, stonemasons, and tile setters',
 '505': 'Cabinetmakers',
 '510': 'Carpenters',
 '511': 'Cement and concrete finishers',
 '512': 'Compositors and typesetters',
 '513': 'Cranemen, derrickmen, and hoistmen',
 '514': 'Decorators and window dressers',
 '515': 'Electricians',
 '520': 'Electrotypers and stereotypers',
 '521': 'Engravers, except photoengravers',
 '522': 'Excavating, grading, and road machinery operators',
 '523': 'Foremen (n.e.c.)',
 '524': 'Forgemen and hammermen',
 '525': 'Furriers',
 '530': 'Glaziers',
 '531': 'Heat treaters, annealers, temperers',
 '532': 'Inspectors, scalers, and graders, log and lumber',
 '533': 'Inspectors (n.e.c.)',
 '534': 'Jewelers, watchmakers, goldsmiths, and silversmiths',
 '535': 'Job setters, metal',
 '540': 'Linemen and servicemen, telegraph, telephone, and power',
 '541': 'Locomotive engineers',
 '542': 'Locomotive firemen',
 '543': 'Loom fixers',
 '544': 'Machinists',
 '545': 'Mechanics and repairmen, airplane',
 '550': 'Mechanics and repairmen, automobile',
 '551': 'Mechanics and repairmen, office machine',
 '552': 'Mechanics and repairmen, radio and television',
 '553': 'Mechanics and repairmen, railroad and car shop',
 '554': 'Mechanics and repairmen (n.e.c.)',
 '555': 'Millers, grain, flour, feed, etc.',
 '560': 'Millwrights',
 '561': 'Molders, metal',
 '562': 'Motion picture projectionists',
 '563': 'Opticians and lens grinders and polishers',
 '564': 'Painters, construction and maintenance',
 '565': 'Paperhangers',
 '570': 'Pattern and model makers, except paper',
 '571': 'Photoengravers and lithographers',
 '572': 'Piano and organ tuners and repairmen',
 '573': 'Plasterers',
 '574': 'Plumbers and pipe fitters',
 '575': 'Pressmen and plate printers, printing',
 '580': 'Rollers and roll hands, metal',
 '581': 'Roofers and slaters',
 '582': 'Shoemakers and repairers, except factory',
 '583': 'Stationary engineers',
 '584': 'Stone cutters and stone carvers',
 '585': 'Structural metal workers',
 '590': 'Tailors and tailoresses',
 '591': 'Tinsmiths, coppersmiths, and sheet metal workers',
 '592': 'Tool makers, and die makers and setters',
 '593': 'Upholsterers',
 '594': 'Craftsmen and kindred workers (n.e.c.)',
 '595': 'Members of the armed services',
 '600': 'Apprentice auto mechanics',
 '601': 'Apprentice bricklayers and masons',
 '602': 'Apprentice carpenters',
 '603': 'Apprentice electricians',
 '604': 'Apprentice machinists and toolmakers',
 '605': 'Apprentice mechanics, except auto',
 '610': 'Apprentice plumbers and pipe fitters',
 '611': 'Apprentices, building trades (n.e.c.)',
 '612': 'Apprentices, metalworking trades (n.e.c.)',
 '613': 'Apprentices, printing trades',
 '614': 'Apprentices, other specified trades',
 '615': 'Apprentices, trade not specified',
 '620': 'Asbestos and insulation workers',
 '621': 'Attendants, auto service and parking',
 '622': 'Blasters and powdermen',
 '623': 'Boatmen, canalmen, and lock keepers',
 '624': 'Brakemen, railroad',
 '625': 'Bus drivers',
 '630': 'Chainmen, rodmen, and axmen, surveying',
 '631': 'Conductors, bus and street railway',
 '632': 'Deliverymen and routemen',
 '633': 'Dressmakers and seamstresses, except factory',
 '634': 'Dyers',
 '635': 'Filers, grinders, and polishers, metal',
 '640': 'Fruit, nut, and vegetable graders, and packers, except factory',
 '641': 'Furnacemen, smeltermen and pourers',
 '642': 'Heaters, metal',
 '643': 'Laundry and dry cleaning operatives',
 '644': 'Meat cutters, except slaughter and packing house',
 '645': 'Milliners',
 '650': 'Mine operatives and laborers',
 '660': 'Motormen, mine, factory, logging camp, etc.',
 '661': 'Motormen, street, subway, and elevated railway',
 '662': 'Oilers and greaser, except auto',
 '670': 'Painters, except construction or maintenance',
 '671': 'Photographic process workers',
 '672': 'Power station operators',
 '673': 'Sailors and deck hands',
 '674': 'Sawyers',
 '675': 'Spinners, textile',
 '680': 'Stationary firemen',
 '681': 'Switchmen, railroad',
 '682': 'Taxicab drivers and chauffers',
 '683': 'Truck and tractor drivers',
 '684': 'Weavers, textile',
 '685': 'Welders and flame cutters',
 '690': 'Operative and kindred workers (n.e.c.)',
 '700': 'Housekeepers, private household',
 '710': 'Laundresses, private household',
 '720': 'Private household workers (n.e.c.)',
 '730': 'Attendants, hospital and other institution',
 '731': 'Attendants, professional and personal service (n.e.c.)',
 '732': 'Attendants, recreation and amusement',
 '740': 'Barbers, beauticians, and manicurists',
 '750': 'Bartenders',
 '751': 'Bootblacks',
 '752': 'Boarding and lodging house keepers',
 '753': 'Charwomen and cleaners',
 '754': 'Cooks, except private household',
 '760': 'Counter and fountain workers',
 '761': 'Elevator operators',
 '762': 'Firemen, fire protection',
 '763': 'Guards, watchmen, and doorkeepers',
 '764': 'Housekeepers and stewards, except private household',
 '770': 'Janitors and sextons',
 '771': 'Marshals and constables',
 '772': 'Midwives',
 '773': 'Policemen and detectives',
 '780': 'Porters',
 '781': 'Practical nurses',
 '782': 'Sheriffs and bailiffs',
 '783': 'Ushers, recreation and amusement',
 '784': 'Waiters and waitresses',
 '785': 'Watchmen (crossing) and bridge tenders',
 '790': 'Service workers, except private household (n.e.c.)',
 '810': 'Farm foremen',
 '820': 'Farm laborers, wage workers',
 '830': 'Farm laborers, unpaid family workers',
 '840': 'Farm service laborers, self-employed',
 '910': 'Fishermen and oystermen',
 '920': 'Garage laborers and car washers and greasers',
 '930': 'Gardeners, except farm and groundskeepers',
 '940': 'Longshoremen and stevedores',
 '950': 'Lumbermen, raftsmen, and woodchoppers',
 '960': 'Teamsters',
 '970': 'Laborers (n.e.c.)',
 '979': 'Not yet classified',
 '980': 'Keeps house/housekeeping at home/housewife',
 '981': 'Imputed keeping house (1850-1900)',
 '982': 'Helping at home/helps parents/housework',
 '983': 'At school/student',
 '984': 'Retired',
 '985': 'Unemployed/without occupation',
 '986': 'Invalid/disabled w/ no occupation reported',
 '987': 'Inmate',
 '990': 'New Worker',
 '991': 'Gentleman/lady/at leisure',
 '995': 'Other non-occupational response',
 '997': 'Occupation missing/unknown',
 '999': 'N/A (blank)'}